# Sort Documents into Individual Folders

In [1]:
from pathlib import Path
import tarfile

import lxml.etree as ET
import pandas as pd
import tqdm

# Grab the TarFiles in Directory

In [2]:
tarfiles = list(Path("../pubtator_abstracts").rglob("*gz"))
tarfiles[0:2]

[PosixPath('../pubtator_abstracts/BioCXML.3.gz'),
 PosixPath('../pubtator_abstracts/BioCXML.8.gz')]

# Load the document Metadata

In [3]:
pmc_metadata_df = pd.read_csv(
    "../exploratory_data_analysis/output/pmc_metadata.tsv.xz", sep="\t"
)
pmc_metadata_df.head()

/home/danich1/anaconda3/envs/biovectors/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,batch_folder,batch_file,doc_id,doi,pmc,pmid,section,published_year
0,BioCXML.3.gz,output/BioCXML/10003.bioc.xml,3616644,10.1021/ja307276z,3616644,22946616.0,ABSTRACT|DISCUSS|FIG|INTRO|METHODS|REF|RESULTS...,2012
1,BioCXML.3.gz,output/BioCXML/10003.bioc.xml,3616678,10.1097/ADT.0b013e31815c2ded,3616678,23565053.0,ABSTRACT|DISCUSS|INTRO|METHODS|REF|RESULTS|TAB...,2008
2,BioCXML.3.gz,output/BioCXML/10003.bioc.xml,3616697,10.1093/nar/gkt072,3616697,23396444.0,ABSTRACT|ACK_FUND|CONCL|FIG|INTRO|METHODS|REF|...,2013
3,BioCXML.3.gz,output/BioCXML/10003.bioc.xml,3616698,10.1093/nar/gkt006,3616698,23335781.0,ABSTRACT|ACK_FUND|DISCUSS|FIG|INTRO|METHODS|RE...,2013
4,BioCXML.3.gz,output/BioCXML/10003.bioc.xml,3616699,10.1093/nar/gkt030,3616699,23393190.0,ABSTRACT|ACK_FUND|DISCUSS|FIG|INTRO|METHODS|RE...,2013


In [5]:
for tar_obj in tarfiles:
    open_tar = tarfile.open(str(tar_obj))

    while True:
        try:
            batch_file = open_tar.next()

            if batch_file is None:
                break

            doc_generator = ET.iterparse(
                open_tar.extractfile(batch_file), tag="document", recover=True
            )

            for event, doc_obj in tqdm.tqdm(doc_generator):
                year = doc_obj.xpath(
                    "passage[contains(infon[@key='section_type']/text(), 'TITLE')]/infon[@key='year']/text()"
                )

                if len(year) < 1 or int(year[0]) < 2000:
                    continue

                year_folder = Path("output") / Path(year[0])
                if not year_folder.exists():
                    year_folder.mkdir()

                pmid = doc_obj.xpath("passage/infon[@key='article-id_pmid']/text()")

                with open(f"output/abstract_output/{year[0]}/{pmid[0]}.xml", "wb") as f:
                    f.write(ET.tostring(doc_obj, pretty_print=True, encoding="utf-8"))

        except tarfile.ReadError:
            print(f"broke out of {str(tar_obj)}")
            break

100it [00:00, 282.80it/s]
100it [00:00, 326.26it/s]
100it [00:00, 260.76it/s]
100it [00:00, 275.97it/s]
99it [00:00, 269.33it/s]
99it [00:00, 315.18it/s]
100it [00:00, 330.24it/s]
100it [00:00, 272.97it/s]
99it [00:00, 291.38it/s]
100it [00:00, 280.77it/s]
100it [00:00, 299.17it/s]
74it [00:00, 302.39it/s]
100it [00:00, 298.78it/s]
100it [00:00, 324.54it/s]
100it [00:00, 484.78it/s]
80it [00:00, 325.23it/s]
99it [00:00, 275.41it/s]
100it [00:00, 275.01it/s]
100it [00:00, 330.60it/s]
99it [00:00, 265.13it/s]
100it [00:00, 272.95it/s]
100it [00:00, 513.27it/s]
100it [00:00, 298.15it/s]
99it [00:00, 269.06it/s]
100it [00:00, 282.60it/s]
100it [00:00, 12935.40it/s]
100it [00:00, 244.39it/s]
100it [00:00, 313.67it/s]
100it [00:00, 276.17it/s]
100it [00:00, 355.05it/s]
100it [00:00, 263.99it/s]
100it [00:00, 314.66it/s]
100it [00:00, 354.78it/s]
100it [00:00, 342.04it/s]
98it [00:00, 312.94it/s]
100it [00:00, 294.50it/s]
100it [00:00, 260.81it/s]
100it [00:00, 414.14it/s]
99it [00:00, 273.07

100it [00:00, 121.94it/s]
100it [00:00, 266.28it/s]
100it [00:00, 133.74it/s]
99it [00:00, 290.87it/s]
100it [00:00, 340.84it/s]
100it [00:00, 289.09it/s]
100it [00:00, 307.92it/s]
100it [00:00, 348.63it/s]
100it [00:00, 354.64it/s]
100it [00:00, 126.88it/s]
100it [00:01, 87.35it/s]
100it [00:00, 152.22it/s]
100it [00:00, 248.25it/s]
100it [00:00, 1062.40it/s]
100it [00:00, 234.06it/s]
100it [00:00, 173.45it/s]
99it [00:00, 232.39it/s]
100it [00:00, 265.37it/s]
100it [00:00, 286.49it/s]
94it [00:00, 273.99it/s]
100it [00:00, 298.10it/s]
99it [00:00, 136.81it/s]
99it [00:00, 361.46it/s]
100it [00:00, 113.02it/s]
100it [00:00, 335.07it/s]
99it [00:00, 311.10it/s]
99it [00:00, 323.28it/s]
100it [00:00, 375.69it/s]
100it [00:00, 344.19it/s]
99it [00:00, 136.76it/s]
99it [00:00, 206.78it/s]
100it [00:00, 104.03it/s]
100it [00:00, 256.69it/s]
100it [00:00, 291.32it/s]
100it [00:00, 280.01it/s]
100it [00:00, 654.29it/s]
100it [00:00, 304.49it/s]
100it [00:00, 295.80it/s]
99it [00:00, 288.84it

98it [00:00, 278.33it/s]
100it [00:00, 282.22it/s]
99it [00:00, 276.87it/s]
99it [00:00, 345.25it/s]
100it [00:00, 366.45it/s]
100it [00:00, 260.50it/s]
100it [00:00, 277.47it/s]
100it [00:00, 141.09it/s]
100it [00:01, 76.82it/s]
99it [00:00, 168.99it/s]
100it [00:00, 249.45it/s]
100it [00:00, 301.43it/s]
100it [00:00, 405.08it/s]
100it [00:00, 286.12it/s]
100it [00:00, 315.35it/s]
100it [00:00, 309.45it/s]
100it [00:00, 208.61it/s]
100it [00:00, 303.81it/s]
100it [00:00, 336.76it/s]
100it [00:00, 249.58it/s]
99it [00:00, 287.18it/s]
100it [00:00, 252.46it/s]
100it [00:00, 118.75it/s]
100it [00:00, 202.53it/s]
99it [00:00, 287.47it/s]
99it [00:00, 354.41it/s]
100it [00:00, 545.48it/s]
100it [00:00, 345.52it/s]
100it [00:00, 304.84it/s]
100it [00:00, 402.89it/s]
100it [00:00, 136.50it/s]
100it [00:00, 421.01it/s]
100it [00:00, 290.32it/s]
99it [00:00, 396.43it/s]
100it [00:00, 286.07it/s]
100it [00:00, 372.13it/s]
99it [00:00, 378.76it/s]
100it [00:01, 77.40it/s]
100it [00:00, 215.55it/

100it [00:00, 300.37it/s]
100it [00:00, 298.05it/s]
100it [00:00, 408.53it/s]
100it [00:00, 279.27it/s]
100it [00:00, 371.39it/s]
100it [00:00, 365.62it/s]
100it [00:00, 390.41it/s]
98it [00:00, 264.59it/s]
100it [00:00, 316.45it/s]
100it [00:01, 95.86it/s]
100it [00:00, 242.02it/s]
100it [00:00, 648.22it/s]
100it [00:00, 313.81it/s]
100it [00:01, 88.39it/s]
99it [00:00, 149.68it/s]
99it [00:00, 230.04it/s]
100it [00:00, 304.91it/s]
100it [00:00, 343.62it/s]
100it [00:00, 309.36it/s]
100it [00:00, 350.35it/s]
100it [00:00, 166.42it/s]
100it [00:00, 438.18it/s]
100it [00:00, 318.95it/s]
100it [00:00, 301.96it/s]
99it [00:00, 369.00it/s]
100it [00:00, 267.36it/s]
100it [00:00, 387.06it/s]
100it [00:00, 348.17it/s]
100it [00:00, 315.93it/s]
99it [00:00, 341.55it/s]
100it [00:01, 89.31it/s]
100it [00:00, 287.46it/s]
99it [00:00, 107.49it/s]
100it [00:00, 183.78it/s]
100it [00:00, 156.06it/s]
99it [00:00, 250.99it/s]
99it [00:00, 165.15it/s]
100it [00:00, 156.80it/s]
99it [00:00, 202.92it/s

100it [00:00, 257.12it/s]
98it [00:00, 303.04it/s]
100it [00:00, 266.39it/s]
89it [00:00, 355.71it/s]
100it [00:00, 147.71it/s]
100it [00:00, 333.66it/s]
100it [00:00, 305.69it/s]
90it [00:00, 271.97it/s]
100it [00:00, 276.96it/s]
100it [00:00, 274.76it/s]
100it [00:00, 185.97it/s]
36it [00:00, 41.78it/s]
34it [00:00, 332.92it/s]
100it [00:00, 338.50it/s]
99it [00:00, 318.76it/s]
44it [00:00, 335.26it/s]
100it [00:00, 154.23it/s]
100it [00:00, 359.44it/s]
100it [00:00, 327.36it/s]
100it [00:00, 309.73it/s]
97it [00:00, 267.10it/s]
47it [00:00, 335.24it/s]
100it [00:00, 330.35it/s]
65it [00:00, 283.99it/s]
90it [00:00, 319.66it/s]
53it [00:00, 339.94it/s]
49it [00:00, 390.43it/s]
81it [00:00, 462.29it/s]
100it [00:00, 265.39it/s]
100it [00:00, 186.89it/s]
73it [00:00, 353.28it/s]
100it [00:01, 77.91it/s]
75it [00:00, 157.05it/s]
4it [00:00, 109.65it/s]
100it [00:00, 248.48it/s]
41it [00:00, 263.22it/s]
100it [00:00, 296.21it/s]
97it [00:00, 219.98it/s]
61it [00:00, 261.40it/s]
100it [00

...

1000it [00:00, 13598.09it/s]
1000it [00:00, 7157.87it/s]
1000it [00:00, 2779.86it/s]
1000it [00:00, 8524.40it/s]
1000it [00:00, 3590.04it/s]
1000it [00:00, 6778.36it/s]
1000it [00:00, 5651.00it/s]
1000it [00:00, 2835.25it/s]
1000it [00:00, 3235.07it/s]
1000it [00:00, 2798.33it/s]
100it [00:01, 85.06it/s]
1000it [00:00, 11657.71it/s]
1000it [00:00, 10022.40it/s]
1000it [00:00, 6825.42it/s]
1000it [00:00, 11965.61it/s]
1000it [00:00, 10157.22it/s]
1000it [00:00, 13470.83it/s]
1000it [00:00, 8003.10it/s]
1000it [00:00, 8300.88it/s]
1000it [00:00, 7620.88it/s]
1000it [00:00, 8925.80it/s]
100it [00:00, 283.46it/s]
1000it [00:00, 4879.92it/s]
99it [00:00, 201.35it/s]
1000it [00:00, 6450.70it/s]
1000it [00:00, 16353.34it/s]
1000it [00:00, 14088.70it/s]
1000it [00:00, 11650.39it/s]
1000it [00:00, 10063.98it/s]
1000it [00:00, 2424.51it/s]
1000it [00:00, 10383.02it/s]
1000it [00:00, 14156.36it/s]
1000it [00:00, 11046.10it/s]
100it [00:00, 382.74it/s]
1000it [00:00, 10929.84it/s]
1000it [00:00, 1

1000it [00:00, 40615.72it/s]
1000it [00:00, 39586.08it/s]
1000it [00:00, 39270.30it/s]
1000it [00:00, 40116.92it/s]
100it [00:00, 327.18it/s]
1000it [00:00, 39386.46it/s]
100it [00:00, 338.20it/s]
1000it [00:00, 39852.76it/s]
1000it [00:00, 37532.25it/s]
1000it [00:00, 5270.84it/s]
1000it [00:00, 37945.48it/s]
1000it [00:00, 38688.56it/s]
1000it [00:00, 38594.57it/s]
1000it [00:00, 37506.74it/s]
1000it [00:00, 40275.24it/s]
1000it [00:00, 39772.27it/s]
99it [00:00, 380.20it/s]
1000it [00:00, 41192.51it/s]
1000it [00:00, 39917.24it/s]
1000it [00:00, 37605.27it/s]
1000it [00:00, 40173.02it/s]
1000it [00:00, 39397.93it/s]
1000it [00:00, 39439.80it/s]
1000it [00:00, 38837.95it/s]
1000it [00:00, 39984.21it/s]
1000it [00:00, 36970.51it/s]
1000it [00:00, 38764.36it/s]
99it [00:00, 140.26it/s]
1000it [00:00, 9016.96it/s]
1000it [00:00, 39525.28it/s]
1000it [00:00, 38719.63it/s]
1000it [00:00, 38773.68it/s]
1000it [00:00, 38074.66it/s]
1000it [00:00, 38493.63it/s]
1000it [00:00, 38945.41it/s]
1

1000it [00:00, 22731.63it/s]
1000it [00:00, 15442.21it/s]
1000it [00:00, 2457.08it/s]
1000it [00:00, 13769.56it/s]
1000it [00:00, 14907.30it/s]
1000it [00:00, 17804.08it/s]
1000it [00:00, 13885.99it/s]
1000it [00:00, 9268.42it/s]
1000it [00:00, 38302.05it/s]
99it [00:00, 200.77it/s]
1000it [00:00, 39821.74it/s]
1000it [00:00, 36386.15it/s]
1000it [00:00, 16163.21it/s]
1000it [00:00, 14410.64it/s]
1000it [00:00, 18347.47it/s]
1000it [00:00, 16268.59it/s]
1000it [00:00, 15550.47it/s]
1000it [00:00, 17812.25it/s]
1000it [00:00, 16746.47it/s]
1000it [00:00, 17172.04it/s]
100it [00:00, 514.16it/s]
1000it [00:00, 16475.84it/s]
1000it [00:00, 16417.86it/s]
1000it [00:00, 3965.54it/s]
1000it [00:00, 18089.81it/s]
1000it [00:00, 19174.57it/s]
1000it [00:00, 24983.20it/s]
1000it [00:00, 14911.17it/s]
1000it [00:00, 14088.08it/s]
1000it [00:00, 15267.00it/s]
1000it [00:00, 22972.04it/s]
100it [00:00, 127.07it/s]
1000it [00:00, 18091.22it/s]
1000it [00:00, 40858.65it/s]
1000it [00:00, 39561.07it/s

1000it [00:00, 16282.99it/s]
1000it [00:00, 15163.73it/s]
1000it [00:00, 17637.35it/s]
99it [00:00, 299.00it/s]
1000it [00:00, 18778.31it/s]
1000it [00:00, 15612.58it/s]
1000it [00:00, 16967.04it/s]
1000it [00:00, 15741.43it/s]
1000it [00:00, 17189.77it/s]
1000it [00:00, 17977.31it/s]
1000it [00:00, 15987.74it/s]
1000it [00:00, 17294.82it/s]
1000it [00:00, 16535.21it/s]
1000it [00:00, 16294.25it/s]
100it [00:00, 226.19it/s]
1000it [00:00, 11594.51it/s]
1000it [00:00, 10682.31it/s]
1000it [00:00, 2538.98it/s]
1000it [00:00, 16587.65it/s]
1000it [00:00, 14572.46it/s]
1000it [00:00, 17897.99it/s]
1000it [00:00, 16306.35it/s]
1000it [00:00, 15746.22it/s]
1000it [00:00, 24637.88it/s]
1000it [00:00, 20743.65it/s]
100it [00:00, 286.13it/s]
1000it [00:00, 17323.10it/s]
1000it [00:00, 16850.41it/s]
1000it [00:00, 16752.62it/s]
1000it [00:00, 19969.17it/s]
1000it [00:00, 17630.76it/s]
1000it [00:00, 18275.68it/s]
1000it [00:00, 18982.62it/s]
1000it [00:00, 28562.21it/s]
1000it [00:00, 18301.51it

100it [00:00, 300.34it/s]
99it [00:00, 301.72it/s]
100it [00:00, 281.73it/s]
99it [00:00, 282.82it/s]
100it [00:00, 202.56it/s]
100it [00:00, 114.60it/s]
100it [00:01, 85.95it/s]
100it [00:00, 111.75it/s]
99it [00:00, 189.46it/s]
100it [00:00, 195.32it/s]
100it [00:00, 157.66it/s]
100it [00:01, 78.69it/s]
100it [00:00, 130.50it/s]
100it [00:01, 94.88it/s]
100it [00:00, 122.90it/s]
99it [00:00, 155.49it/s]
99it [00:00, 216.28it/s]
100it [00:01, 65.43it/s]
100it [00:00, 269.15it/s]
100it [00:00, 214.24it/s]
100it [00:00, 220.06it/s]
99it [00:00, 172.72it/s]
99it [00:00, 270.32it/s]
100it [00:00, 247.69it/s]
99it [00:00, 167.70it/s]
100it [00:00, 216.06it/s]
100it [00:00, 182.63it/s]
99it [00:00, 155.10it/s]
100it [00:00, 134.50it/s]
99it [00:00, 212.93it/s]
100it [00:00, 152.90it/s]
100it [00:01, 65.16it/s]
99it [00:00, 151.10it/s]
100it [00:00, 159.80it/s]
100it [00:00, 110.46it/s]
100it [00:00, 238.39it/s]
99it [00:00, 118.90it/s]
100it [00:00, 269.91it/s]
99it [00:00, 178.98it/s]
100i